# Already exists

This technique is called "electrorotation".

\cite{Dielectric1988a}


\cite{Electrorotation1988}

> In this article we are concerned with the induction of rotation in microscopic
particles by use of a rotating electric field. The rotation is not synchronous
with the field; in fact the particle speed is dependent on the square of the
applied field strength [ 1 ] and reaches a maximum of perhaps 10 Hz, although
field frequencies between 1 kHz and I MHz are typical [2, 3].


> . Culture-based viability techniques
are also slow, typically taking more than 1 week (Gaspard
et al. 1996). Electrorotation, by comparison, is a quick and
simple technique, with a sample preparation time of <15 min
and measurement time of a similar period.



Optical centrifuges rotate the polarization axis of an electromagnetic field, spinning up polarized molecules until centrifugal forces rip them apart. This has been demonstrated with simple molecules [Villeneuve 2000], [Karczmarek 1999].

The oscillating electric field in our technique induces a slight dipole polarization on the virus, giving us a foothold through which a torque could be applied. 

Modulating the polarization axis of microwave transmitters is an established technique, and can be accomplished in a number of ways:

- Switching the antenna feedport location using PiN diodes [Medina-Sanchez], [Silveira 2018] (limiting the switching frequency to a few KHz)
- Modulating the phase shift between feed-ports so as to produce circular polarization
- Mechanically rotating the antenna
- Using four or more standard linearly-polarized antennas at various angles, modulating the power to buffer amplifiers.

There is also the concern that biological tissues could exhibit the same effect.

This sounds plausible, but then most wrong ideas do.

A first-order Fermi estimate:

In [51]:
from math import pi, sqrt

from pytexit import py2tex

We will need the dipole moment, and for this we need the charge separation. 

Here we make our first assumption; that the charge separation is the same as the mechanical displacement. This is only a good approximation if each charge is strongly bound to a site on the virus, and is almost certainly wrong.
Computing the true polarizability seems quite involved.

Born-Oppenheimer surfaces, which some molecular dynamics programs can account for [GROMACS 2020]. 



We the amplitude from [Yang 2015], eq. 7,

$$A = \frac{q E_0}{\mu \sqrt{((2 \pi f_{res})^2 - (2 \pi f_{excite})^2)^2 + \left(\frac{(2 \pi f_{res})(2 \pi f_{excite})}{Q}\right)^2}}$$

and we assume that the charge separation is twice the amplitude - $d_q = 2A$. 

Next we need to know how much energy is lost to friction. Here comes our next, even more unreasonable assumption: that the friction coefficient for rotation is equal to that of the dipole translation. This allows us to extract the friction coefficient from the vibrational Q factor obtained by [Yang 2015]. 

If the former was unfounded, this one is kooky. Unlike in translational vibration, where the solvent must be inertially displaced, according to [Steele 1963], in the molecular regime a perfect sphere can theoretically undergo perfect slip with the surrounding solvent; friction can only be introduced if there is a change in potential energy between different positions, and a uniform sphere has equal potential energy in every position.

This also means that for molecular dynamics simulation, implicit solvation techniques using, for instance, Langevin integrators with fixed damping coefficients are not a good match for this problem.

Via [Hu 1974], a small degree of oblateness or prolateness can affect the rotational friction coefficient decisively. But if the virus tends to rotate along its minimum friction axis, this effect should not be pronounced.

The virus's surface is not smooth; spike proteins and other clutter may also slow its rotation.

A drag force coefficient

$$D = \frac{\sqrt{\mu k}}{Q}$$

$$2 \pi f_{res} = \sqrt{k/\mu}$$


$$k = (2 \pi f_{res})^2 \mu$$


$$D = \frac{\sqrt{\mu^2 (2 \pi f_{res})^2 }}{Q}$$


assuming that the drag force is exclusively applied to the outer envelope,


$$F_{tangential} = D v$$


$$v = d_{virus} \pi f_{rot}$$

$$\tau_{friction} = D \pi f_{rot}(d_{virus}^2)/2$$

$$p = q \cdot d_q$$

$$\tau_{dipole} = p E$$

$$p E = D \pi f_{rot} (d_{virus}^2)/2$$

$$f_{rot} = \frac{p E}{D (\pi) (d_{virus}^2)/2}$$


where

    D = drag force coefficient
    p = dipole moment
    q = separated charge (1/2 total charge), C
    M = virus mass 
    mu = reduced mass
    k = spring constant of mass-spring system
    Q = virus quality factor
    d_virus = virus envelope diameter, meters
    E = electric field, V/m
    f_res = resonant frequency of linear, dipole mode, Hz
    f_rot = frequency of rotational mode, Hz
    v = tangential velocity at envelope surface
    



In [91]:
E = 200.0 # V/m
e = 1.602e-19
q = 3.0*1e7*e #C

d_virus = 120e-9

M = 2.9e-19 #kg
#mu = ((M/2.0)**2.0)/(2.0*(M/2.0))

mu = 2.4e-20 # using [Yang 2015]'s reduced mass 

Q = 2.0
f_res = 8e9
f_excite = 8e9

delta_f = ((2*pi*f_res)**2.0-(2*pi*f_excite)**2.0)**2.0
A = q * E / (mu * sqrt((delta_f + ((((2.0*pi*f_res)*(2.0*pi*f_excite))/Q)**2.0))))   
d_q = 2.0*A 
print("Charge displacement: {:.0f} pm".format(A/1e-12))


p = q*d_q

D = sqrt((mu**2.0)*((2.0*pi*f_res)**2.0))/Q

print(D)

f_rot = (p*E)/(D*pi*((d_virus**2.0)/2.0))

print("Peak rotational frequency: {:.0e} Hz".format(f_rot))

Charge displacement: 32 pm
6.031857894892403e-10
Peak rotational frequency: 4e+03 Hz


From [Holsapple 2008], the upper bound on the limiting angular frequency for a homegenous elastic sphere is at most $2.236 \ \bar \omega$: 

$$\bar \omega = \sqrt{3}\sqrt{\frac{Y}{\rho a^2}}$$

with radius a, mass density $\rho$, yield stress Y.

In [92]:
r_virus = (d_virus / 2.0)
V = (4.0/3.0) * (pi*(r_virus**3.0))
rho = M / V 

Y = 0.141e6 # Pa - from [Yang 2015]

w_rot_limit = 2.236 * sqrt(3.0) * sqrt(Y / (rho * ((r_virus)**2.0)))
f_rot_limit = w_rot_limit / (2.0*pi)

print("Limit frequency: {:.0e} Hz".format(f_rot_limit))

Limit frequency: 2e+08 Hz


The damping, yield stress, and charge separation would each have to be off by a factor of 50 for this to be a viable technique, which is probably unlikely.

The rotation-induced signal modulation could still be useful for detecting the existence of the virus; but for inactivation, better techniques exist.

 

-


In [63]:
I = 2.0/5.0 * M * ((0.5*d_virus)**2.0)

tau = p*E #torque

x = tau/I # 10
w_rot_limit/x

9.276557494642485e-06

In [90]:
#http://spiff.rit.edu/classes/phys283/lectures/forced_ii/forced_ii.html

time_constant = ((2.0*mu)/((2.0*pi*8e9*mu)/Q))

time_constant / (1.0/8e9)

0.6366197723675813